In [1]:
import pandas as pd
from numpy import nan
from string import punctuation
punctuation += '\"“”‘’—-–'

def kill_punctuations_capitals(string): 
    string = string.replace("’s","") 
    translator = str.maketrans("","",punctuation) 
    return string.lower().translate(translator)

def check_real(string, real_check_list):
    j = 0
    for i in real_check_list:
        j += kill_punctuations_capitals(string).find(i)
    return j > -len(real_check_list)

def count_frequency(text):
    def kill_punctuations_capitals(text): 
        global punctuation
        punctuation += '\"“”‘’—-–·`'
        translator = str.maketrans("","",punctuation) 
        return text.replace("’s","").translate(translator).lower().split() 
        #去除‘s，去除標點，將大寫字母轉化為小寫，列出單詞

    def words_frequency(list):  #去除stopwords統計一篇文章中的frequency
        with open ('stopword.txt','r') as s:
            list_stop_words = s.read().split()  #讀取stoplist
        return {m: list.count(m) for m in [
                m for m in list if m not in list_stop_words]}

    def rank_frequency(dict): 
        dict_frequency_rank={}
        rank = sorted(
            dict.items(), 
            key=lambda item: item[1], 
            reverse=True) 
        #將字典轉化爲二元數組，並根據字典中value排序
        for m in range(0,len(rank)):
            dict_frequency_rank.update({
                rank[m][0]:rank[m][1]}) 
            #再將字典重新整合起來
        return dict_frequency_rank
    
    return rank_frequency(
        words_frequency(
            kill_punctuations_capitals(text)))

In [10]:
# 筛选真正的记者职位
df_all_jobs = pd.read_csv("0 jobs.csv",header=0)

real_check_list = [
    'data',
    'digital',
    'visual',
    'program'
"investigat",
"analys"
]

df_all_jobs['Real'] = [
    check_real(i[1][0], real_check_list) for i in df_all_jobs.iterrows()] 
# .iterrows()可以根据index遍历, df_all_jobs.iterrows()是[(0,series),(1,series)...], 
# i是(0,series), i[1]是series, i[1][0]是Title

df_selected_jobs = df_all_jobs[
    df_all_jobs['Real']==True].sort_values(
        by='Post_Date',ascending=False)

df_selected_jobs.to_csv(
    '1.5 new-selected-jobs.csv',
    na_rep = nan,
    index=False)

In [67]:
#统计关键词的frequency
key_words = [
    'data',
    'dataset',
    'data analysis',]

df_selected_jobs = pd.read_csv("1 new-selected-jobs.csv")

df_key_frequency = df_selected_jobs.drop(
    'Description',
    axis=1,
    inplace=False)  #False 代表不在原数据上保存

list_jobs_frequency = [
    count_frequency(i) for i in df_selected_jobs['Description']]

df_key_frequency = df_key_frequency.join(
    pd.DataFrame([[
        m.get(i) or nan for m in list_jobs_frequency] 
        for i in key_words],
        index=key_words).T)

df_key_frequency.to_csv(
    '2 jobs-{}-frequency.csv'.format('-'.join(key_words)),
    na_rep = nan,
    index=False)

CPU times: user 1.29 s, sys: 40 ms, total: 1.33 s
Wall time: 1.38 s


In [3]:
#统计所有的frequency
df_selected_jobs = pd.read_csv("1 new-selected-jobs.csv")
dict_frequency = count_frequency(df_selected_jobs['Description'].sum())
df_frequency = pd.DataFrame(dict_frequency,index=[0]).T
df_frequency.index=range(1,len(df_frequency) + 1)
df_frequency.to_csv('3 description-frequency-rank.csv')